In [86]:
from mlxtend.frequent_patterns import fpgrowth
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from IPython.display import display
import re
from collections import Counter

In [2]:
dataset = pd.read_csv("/dados01/workspace/ufmg.f01dcc/f03/data/output/druid/items_clusters_train.csv.zip", sep=";")

In [ ]:
dataset["cluster_id"] = dataset.cluster.apply(lambda g: str(g).split("_")[0])
grupos = dataset.groupby("cluster_id")["cluster_id"].count().sort_values(ascending=False)
grupos = pd.DataFrame(grupos).rename(columns={"cluster_id": "count"}).reset_index()

In [91]:
# Transforma em tokens e remove os que possuem menos de 3 caracteres
is_num = re.compile("^[0-9]+$")
groups_description = {}
for grupo in grupos.cluster_id.to_list()[:30]:
    print(f"Gen description for group: {grupo}")
    items = dataset.loc[dataset.cluster.str.startswith(grupo)]
    description = items.description.drop_duplicates()
    description = description.apply(lambda v: [tk for tk in str(v).split() if not is_num.match(tk) and len(tk) > 1])
    groups_description[grupo] = description

# Controi a árvore
groups_tree = {}
for grupo, descriptions in groups_description.items():
    te = TransactionEncoder()
    te_ary = te.fit(descriptions).transform(descriptions)
    groups_tree[grupo] = pd.DataFrame(te_ary, columns=te.columns_)

Gen description for group: papel
Gen description for group: luva
Gen description for group: filtro
Gen description for group: broca
Gen description for group: tubo
Gen description for group: pneu
Gen description for group: cabo
Gen description for group: fita
Gen description for group: oleo
Gen description for group: tinta
Gen description for group: lampada
Gen description for group: sonda
Gen description for group: pasta
Gen description for group: item
Gen description for group: joelho
Gen description for group: bucha
Gen description for group: parafuso
Gen description for group: fio
Gen description for group: pincel
Gen description for group: saco
Gen description for group: disjuntor
Gen description for group: porta
Gen description for group: livro
Gen description for group: cola
Gen description for group: cartucho
Gen description for group: caneta
Gen description for group: rolamento
Gen description for group: jogo
Gen description for group: agulha
Gen description for group: manguei

In [92]:
# Calcular suporte em até 60% do dataset
for grupo, tree in groups_tree.items():
    print(grupo)
    df = fpgrowth(tree, min_support=0.15, use_colnames=True)
    df.sort_values(by="support", ascending=False, inplace=True)
    print(df)
    print()

papel
     support             itemsets
6   0.994656              (papel)
7   0.301323                (cor)
16  0.301051         (cor, papel)
0   0.299458                (com)
8   0.298428         (papel, com)
1   0.279307                 (cm)
9   0.278686          (cm, papel)
2   0.232225              (folha)
11  0.231875       (papel, folha)
3   0.176457                 (a4)
13  0.176301          (papel, a4)
5   0.167071             (branco)
15  0.166605      (papel, branco)
4   0.154363             (pacote)
14  0.154247      (papel, pacote)
10  0.150555         (folha, com)
12  0.150283  (papel, folha, com)

luva
     support              itemsets
0   0.999629                (luva)
2   0.255067                 (com)
9   0.254998           (com, luva)
7   0.244585               (latex)
14  0.244538         (luva, latex)
1   0.231087                  (mm)
8   0.231017            (luva, mm)
4   0.209448             (tamanho)
11  0.209333       (tamanho, luva)
5   0.184679              

    support          itemsets
3  0.919147           (porta)
0  0.246205              (cm)
4  0.231922       (cm, porta)
1  0.167749             (com)
2  0.163704         (madeira)
5  0.150170  (madeira, porta)

livro
    support                itemsets
1  0.999257                 (livro)
3  0.286785                 (autor)
8  0.286623          (livro, autor)
2  0.267759                (editor)
5  0.267662         (livro, editor)
6  0.187131         (autor, editor)
7  0.187098  (livro, autor, editor)
0  0.150693                   (com)
4  0.150628            (com, livro)

cola
    support        itemsets
4  0.746572          (cola)
0  0.257660           (com)
1  0.215040          (para)
2  0.212541         (colar)
3  0.194953        (branco)
5  0.194494     (com, cola)
7  0.188529  (cola, branco)
6  0.170992    (cola, para)

cartucho
     support                         itemsets
4   0.999788                       (cartucho)
0   0.524621                             (hp)
9   0.524494     